In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random

# 数据增强和预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小为 224x224
    transforms.Grayscale(num_output_channels=3),  # 将灰度图像转换为三通道
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 标准化，通常使用预训练模型的均值和标准差
])

# 设置全局随机种子
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

def worker_init_fn(worker_id):
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=worker_init_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, )

# 加载预训练的 ResNet-18 模型
model = models.resnet18(pretrained=True)

# 冻结模型的所有层，以避免在训练过程中更新它们的权重
for param in model.parameters():
    param.requires_grad = False

# 替换 ResNet-18 的最后一层
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 将输出层改为10个分类节点（对应MNIST的10个类别）

# 将模型移到GPU（如果可用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/Users/wuhonglei1/Desktop/人工智能/python/dh_ai/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/wuhonglei1/Desktop/人工智能/python/dh_ai/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# 训练模型
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_index, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
        if (batch_index + 1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                  % (epoch + 1, num_epochs, batch_index + 1, len(train_dataset) // 32, loss.item()))

Epoch [1/5], Iter [100/1875] Loss: 1.1246
Epoch [1/5], Iter [200/1875] Loss: 0.6718
Epoch [1/5], Iter [300/1875] Loss: 0.5726
Epoch [1/5], Iter [400/1875] Loss: 0.2991
Epoch [1/5], Iter [500/1875] Loss: 0.5009
Epoch [1/5], Iter [600/1875] Loss: 0.3825
Epoch [1/5], Iter [700/1875] Loss: 0.5031
Epoch [1/5], Iter [800/1875] Loss: 0.1827
Epoch [1/5], Iter [900/1875] Loss: 0.2697
Epoch [1/5], Iter [1000/1875] Loss: 0.2628


In [ ]:
# 在测试集上评估模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')